# 00 - Prerequisites

1. Unzip the 7z file. Manual task.
2. Clear the parquet file directory

In [1]:
import os

# Define the name of the directory to be created
dirs = ["data","data/in","data/out","data/out/table"]

for directory in dirs:
    if os.path.exists(directory):
        print(f"{directory} already exists")
    else:
        os.mkdir(directory)
        print(f"Directory '{directory}' created")

data already exists
data/in already exists
data/out already exists
data/out/table already exists


### Run this to cleanup/reset the out folder

In [2]:
import shutil

# Define the path to the directory containing Parquet files
file_name="trip_data"
raw_table_name="raw_trip_data_table"
parq_output_files=["data/out/"+file_name+".parquet",
                  "data/out/table/"+raw_table_name]

# Check if the directory exists
for file in parq_output_files:
    if os.path.exists(file):
        shutil.rmtree(file)
        print(f'{file} has been deleted.')
    else:
        print(f'{file} does not exist.')

data/out/trip_data.parquet does not exist.
data/out/table/raw_trip_data_table does not exist.


# 01 - Prepare the data

## 1.1 Load csv files to hive table. Let's call it Raw layer

Raw Layer contains the base format of data. We only add bucketing and partitioning to it.

### 1.1.1 Spark Setup

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BDM2024 - Project01") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.driver.memory", "8g")\
    .enableHiveSupport() \
    .config("spark.sql.warehouse.dir", "data/out/table") \
    .getOrCreate()


In [5]:
# Enable below if you are resetting the docker image too.
# This prouces a gazillion logs and you will scratch your head why are you losing so much disk space
#spark.sparkContext.setLogLevel("DEBUG")

### 1.1.2. List the files to be ingested.

In [6]:
file_name="trip_data"
file_prefix="data/in/"+file_name+"_"
# List of CSV files
#csv_file_paths = [file_prefix +"test1.csv",file_prefix +"test2.csv"]
csv_file_paths = [file_prefix + "1.csv" , file_prefix + "2.csv", file_prefix + "3.csv",
                  file_prefix + "4.csv", file_prefix + "5.csv", file_prefix + "6.csv",
                  file_prefix + "7.csv", file_prefix + "8.csv", file_prefix + "9.csv",
                  file_prefix + "10.csv", file_prefix + "11.csv", file_prefix + "12.csv"]

### 1.1.3. Declare the schema to save some resources.

In [7]:
from pyspark.sql.functions import col,to_date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType

column_partitioner="medallion"

raw_file_schema= StructType([
    StructField("medallion", StringType(), True),
    StructField("hack_license", StringType(), True),
    StructField("vendor_id", StringType(), True),
    StructField("rate_code", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_time_in_secs", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True)
])


### 1.1.4 Read the csv files

Add the persists so that the file remains to disk

In [8]:
df = spark.read.csv(csv_file_paths, header=True, schema=raw_file_schema).persist()

### 1.1.5. Create the hive table

Apply `bucketing` and `partitioning` to the table.

In [9]:
bucket_count=400

# To reset the table
spark.sql("DROP TABLE IF EXISTS "+raw_table_name)
df.withColumn("pickup_date", to_date(col("pickup_datetime")))\
    .write\
    .format("parquet")\
    .bucketBy(bucket_count,column_partitioner)\
    .partitionBy("pickup_date")\
    .sortBy("pickup_datetime")\
    .mode("append")\
    .saveAsTable(raw_table_name)
df.unpersist()    
print("Writing parquet file successful.")

Writing parquet file successful.


### 1.1.6 Sanity Checks

Check the details from the Raw Layer table

In [10]:
trip_data_bucketed = spark.table(raw_table_name)

spark.sql("SHOW TABLES").show(truncate=False)
spark.sql("SHOW PARTITIONS "+raw_table_name).show(truncate=False)

+---------+-------------------+-----------+
|namespace|tableName          |isTemporary|
+---------+-------------------+-----------+
|default  |raw_trip_data_table|false      |
+---------+-------------------+-----------+

+----------------------+
|partition             |
+----------------------+
|pickup_date=2013-01-01|
|pickup_date=2013-01-02|
|pickup_date=2013-01-03|
|pickup_date=2013-01-04|
|pickup_date=2013-01-05|
|pickup_date=2013-01-06|
|pickup_date=2013-01-07|
|pickup_date=2013-01-08|
|pickup_date=2013-01-09|
|pickup_date=2013-01-10|
|pickup_date=2013-01-11|
|pickup_date=2013-01-12|
|pickup_date=2013-01-13|
|pickup_date=2013-01-14|
|pickup_date=2013-01-15|
|pickup_date=2013-01-16|
|pickup_date=2013-01-17|
|pickup_date=2013-01-18|
|pickup_date=2013-01-19|
|pickup_date=2013-01-20|
+----------------------+
only showing top 20 rows



## 1.2 Enrich the data with geojson

### 1.2.1. Read geojson file

In [14]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType, LongType, ArrayType

# https://gist.github.com/JulesBelveze/a552e8c53dfd1f46948cbeb32c096611
geojson_data="data/in/nyc-boroughs.geojson"
# Define the custom schema
geojson_schema = StructType([
    StructField("type", StringType(), nullable=True),
    StructField("id", LongType(), nullable=True),
    StructField("properties", StructType([
        StructField("boroughCode", LongType(), nullable=True),
        StructField("borough", StringType(), nullable=True),
        StructField("@id", StringType(), nullable=True)
    ]), nullable=True),
    StructField("geometry", StructType([
        StructField("type", StringType(), nullable=True),
        StructField("coordinates", ArrayType(ArrayType(ArrayType(DoubleType()))), nullable=True)
    ]), nullable=True)
])

# Read GeoJSON file with custom schema
geojson_df = spark.read.schema(geojson_schema).json(geojson_data)

In [15]:
# View the schema
geojson_df.printSchema()

# Show the data
geojson_df.show()
geojson_df = geojson_df.na.drop() #remove the NULL values

# Show the resulting DataFrame
geojson_df.show()

num_rows = geojson_df.count()
print(f"Number of boroughs: {num_rows}")

root
 |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- boroughCode: long (nullable = true)
 |    |-- borough: string (nullable = true)
 |    |-- @id: string (nullable = true)
 |-- geometry: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: double (containsNull = true)

+-------+----+--------------------+--------------------+
|   type|  id|          properties|            geometry|
+-------+----+--------------------+--------------------+
|   NULL|NULL|                NULL|                NULL|
|   NULL|NULL|                NULL|                NULL|
|   NULL|NULL|                NULL|                NULL|
|Feature|   0|{5, Staten Island...|{Polygon, [[[-74....|
|   NULL|NULL|                NULL|                NULL

In [16]:
!pip install shapely

In [19]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, LongType
# Create UDF for finding idle time between two times
def idle_time_ms(start_ms, prev_end_ms):
    if start_ms is None or prev_end_ms is None:
        return 0
    idle_ms = start_ms - prev_end_ms
    threshold_duration_ms = 4 * 60 * 60 * 1000  # 4 hours in milliseconds
    if idle_ms < 0 or idle_ms > threshold_duration_ms:
        return 0
    else:
        return idle_ms
    
# Define as UDF
idle_time_ms_udf = udf(idle_time_ms, LongType())

In [20]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, unix_timestamp, sum, col
from datetime import datetime

# Configure the window

# Define the window specification with partitionBy and orderBy
window_conf = Window.partitionBy(col("medallion")).orderBy(col("pickup_date"), col("pickup_time"))
# Medallion refers to the vehicle, hack_license refers to the driver
window_conf = Window.partitionBy(col("medallion")).orderBy(col("pickup_date"),col("pickup_time"))

# TODO: Add shuffling here
#   ...
taxi_util_data = trip_data_bucketed
# Extract pickup_date and pickup_time from pickup_datetime
taxi_util_data = taxi_util_data.withColumn("pickup_date", to_date(col("pickup_datetime")))
taxi_util_data = taxi_util_data.withColumn("pickup_time", date_format(col("pickup_datetime"), "HH:mm:ss"))

# Add column with pickup datetime of previous
# NB! The default value for the lag might not be correct
taxi_util_data_sorted = taxi_util_data.withColumn("dropoff_datetime_prev", lag(col("dropoff_datetime"), default=datetime.min).over(window_conf))

# Convert to timestamp
taxi_util_data_sorted_ts = taxi_util_data_sorted.withColumn("pickup_ts_ms", unix_timestamp("pickup_datetime") * 1000) \
    .withColumn("dropoff_prev_ts_ms", unix_timestamp("dropoff_datetime_prev") * 1000)

# Calculate idle time per ride
taxi_util_data_idle = taxi_util_data_sorted_ts.withColumn("idle_time_ms", idle_time_ms_udf("pickup_ts_ms", "dropoff_prev_ts_ms"))

# Calculate total idle time per taxi
taxi_util_data_idle_total = taxi_util_data_idle.groupBy(col("medallion")).agg(sum(col("idle_time_ms")))

In [21]:
taxi_util_data_idle_total.show()

+--------------------+-----------------+
|           medallion|sum(idle_time_ms)|
+--------------------+-----------------+
|1109955CCAABCBCE1...|       7789070000|
|18DDC049CC26AE3AA...|       2965980000|
|1D8B19995A8EBDF9B...|       5282700000|
|223670562219093D6...|      10122420000|
|2E7374B2849EB39CD...|       5122665000|
|35B2F21FAF5E53F1E...|      13082705000|
|57E8E649531AB8807...|      13253708000|
|586D9BD604B923DA3...|      12173972000|
|595917A7813CC80DA...|      14438432000|
|59DF6039EC312EE6D...|      11192880000|
|6695FB6E06F7D99F5...|      12821718000|
|72EAFBA3FB9F0507C...|      12918600000|
|73039762E0F4B253E...|      13207440000|
|73F27D8C8C3B37D04...|       7252834000|
|753BC0484097BB236...|      13198400000|
|764CA5AE502C0FEC9...|      12894830000|
|846DFE2D59F6E76EC...|       8048640000|
|87EB479F55B88D47C...|      10627020000|
|963BEE5F306952D20...|      12738530000|
|98C89210E1228F7AE...|       8587942000|
+--------------------+-----------------+
only showing top